In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import xarray as xr
import pingouin as pg
import os
from tqdm import tqdm
from os.path import join as pjoin
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, zscore, wilcoxon
import sys

sys.path.append('..')
import circletrack_behavior as ctb
import pca_ica as ica
import plotting_functions as pf
import place_cells as pc

In [2]:
## Settings
## Create a list of sessions for easy plotting later and for changing column order
session_list = ['Training1', 'Training2', 'Training3', 'Training4', 'Reversal1']
## Set path variables
assembly_path = '../../../../../csstorage/phild/git/MazeProjects/output/assemblies'
calc_path = '../../../../../csstorage/phild/git/MazeProjects/output/processed/'
behav_path = '../../../../../csstorage/phild/git/MazeProjects/output/behav'
registration_path = '../../../../../csstorage/phild/git/MazeProjects/output/registration/'
fig_path = '../../Memory_Updating/EnsembleRemodeling_Resubmission/intermediate_figures/cohort0/'
## Create young and old mouse list
young_mice = ['Fornax', 'Janus', 'Lyra', 'Miranda', 'Naiad', 'Sao', 'Titania']
old_mice = ['Gemini', 'Oberon', 'Puck', 'Umbriel', 'Virgo', 'Ymir', 'Atlas']
## Create list of female or male young mice
young_female_mice = ['Fornax', 'Janus', 'Lyra', 'Sao', 'Titania']
male_young_mice = ['Miranda', 'Naiad']
x_bin_size = 1

In [3]:
## Get fading ensembles for young mice
across_time = False
alpha_old = 0.05
analysis_type = 'max'
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(young_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, assembly_path, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            aligned_behavior = pd.read_feather(pjoin(behav_path, f'{mouse}_{session}.feat'))
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold=None, x_bin_size=x_bin_size, analysis_type=analysis_type, 
                                                                                           zscored=True, alpha_old=alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trial_type='all', 
                                                                                               z_threshold=None, alpha_old=alpha_old, analysis_type=analysis_type)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

 14%|█▍        | 1/7 [00:01<00:11,  1.91s/it]/home/austinbaggetta/.conda/envs/mem_updating/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
/home/austinbaggetta/.conda/envs/mem_updating/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
/home/austinbaggetta/.conda/envs/mem_updating/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
 29%|██▊       | 2/7 [00:04<00:11,  2.22s/it]/home/austinbaggetta/.conda/envs/mem_updating/lib/python3.9/site-packages/statsmodels/stats/multitest.py:164: RuntimeWarning: divide by zero encountered in log1p
  pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))
 43%|████▎     | 3/7 [00:06<00:08,

In [ ]:
## One mouse example
session_one = 'Training4' 
session_two = 'Reversal1'
iterations = 100
direction = 'positive'
alpha = 0.05
percentile = 99
session_of_interest = 'Reversal1'
mouse = 'Fornax'
test = 'pearson'

matched_ensembles = ica.match_ensembles_between_sessions(mouse, registration_path, assembly_path, session_one, session_two,
                                                         direction, test=test, iterations=iterations, alpha=alpha, percentile=percentile)
results_df, fading_ensembles, nonfading_ensembles = ica.determine_proportion_matched(mouse, mouse_ensembles, matched_ensembles, session_of_interest)

In [ ]:
fading_ensembles

In [ ]:
## Loop through all mice to determine the proportion of fading ensembles that are matched between sessions
session_one = 'Training4' 
session_two = 'Reversal1'
iterations = 100
direction = 'positive'
alpha = 0.05
percentile = 99
session_of_interest = 'Reversal1'
test = 'pearson'

combined_fading = pd.DataFrame()
combined_nonfading = pd.DataFrame()
combined_results = pd.DataFrame()
for mouse in tqdm(young_mice):
    print(f'Analyzing {mouse}...')
    matched_ensembles = ica.match_ensembles_between_sessions(mouse, registration_path, assembly_path, session_one, session_two,
                                                         direction, test=test, iterations=iterations, alpha=alpha, percentile=percentile)
    matched_ensembles = matched_ensembles[matched_ensembles['statistic'] > 0.2] ## value determined from 99th percentile of intra-session correlations
    results_df, fading_ensembles, nonfading_ensembles = ica.determine_proportion_matched(mouse, mouse_ensembles, matched_ensembles, session_of_interest)

    ## Combine results across mice
    combined_fading = pd.concat([combined_fading, fading_ensembles])
    combined_nonfading = pd.concat([combined_nonfading, nonfading_ensembles])
    combined_results = pd.concat([combined_results, results_df])

In [ ]:
## Plot the proportion of matched fading and nonfading ensembles across young mice
fading_results = combined_results
fading_results['prop_fading_matched'] = fading_results['num_fading_matched'] / fading_results['num_fading']
fading_results['prop_nonfading_matched'] = fading_results['num_nonfading_matched'] / fading_results['num_nonfading']
fading_per_mouse = fading_results.melt(id_vars=['mouse'], value_vars=['prop_fading_matched', 'prop_nonfading_matched'], 
                                 var_name='ensemble_type', value_name='prop').replace({'prop_fading_matched': 'Fading',
                                                                                       'prop_nonfading_matched': 'Non-fading'})
fading_avg = fading_per_mouse.groupby(['ensemble_type'], as_index=False).agg({'prop': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Proportion Matched')
fig.add_trace(go.Bar(x=fading_avg['ensemble_type'], y=fading_avg['prop']['mean'],
                     error_y=dict(type='data', array=fading_avg['prop']['sem'], thickness=2.5, width=10),
                     marker_color='turquoise', marker_line_color='black', 
                     marker_line_width=2, opacity=0.8, showlegend=False))
for mouse in np.unique(fading_per_mouse['mouse']):
    mouse_data = fading_per_mouse[fading_per_mouse['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mouse_data['ensemble_type'], y=mouse_data['prop'], mode='markers', 
                             marker_color='darkgrey', name=mouse, showlegend=False, marker=dict(line=dict(width=1))))
fig.show()
# fig.write_image(pjoin(fig_path, 'proportion_matched_R_T4.png'))

In [ ]:
cutoff = np.percentile(df['statistic'][df['ensemble_id2'] == 5], percentile)
fig = pf.custom_graph_template(x_title='Pearson Correlation (r)', y_title='Probability')
fig.add_trace(go.Histogram(x=df['statistic'][df['ensemble_id1'] == 5], histnorm='probability', 
                           marker_color='darkgrey'))
fig.add_vline(x=cutoff, line_width=1, line_dash='dash', line_color='red', opacity=1)
fig.show()
print(cutoff)
fig.write_image(pjoin(fig_path, 'test_distribution_bootstrap.png'))

In [ ]:
## Check two ensembles that were matched
mouse = 'Fornax'
session_one = 'Training4'
session_two = 'Reversal1'
mappings = pd.read_feather(pjoin(registration_path, f'{mouse}.feat'))
session_one_assemblies = ica.load_session_assemblies(mouse, assembly_path, session_id=session_one)
session_two_assemblies = ica.load_session_assemblies(mouse, assembly_path, session_id=session_two)
## Get pairs of cells between the two sessions of interest
pairs = mappings.loc[:, [f'{session_one}', f'{session_two}']].dropna()
## Select cells from the cells that are paired between the two sessions
session_one_units = pairs.loc[:, f'{session_one}'].to_numpy()
session_two_units = pairs.loc[:, f'{session_two}'].to_numpy()
a_first = session_one_assemblies.sel(unit_id=session_one_units)
a_second = session_two_assemblies.sel(unit_id=session_two_units)

In [ ]:
fig = pf.stem_plot(a_first['patterns'][11], x_title='Neuron', y_title='Weight')
fig.update_layout(yaxis_range=[-0.3, 0.1])
fig.show()
# fig.write_image(pjoin(fig_path, 'nonfading_ensemble_T4_weights.png'))

In [ ]:
fig = pf.stem_plot(a_second['patterns'][8], member_color='red', x_title='Neuron', y_title='Weight')
fig.update_layout(yaxis_range=[-0.3, 0.1])
fig.show()

### Analysis for old mice

In [ ]:
## Get fading ensembles for old mice
across_time = False
alpha_old = 0.05
analysis_type = 'max'
## Create empty dictionaries
mouse_trends = {}
mouse_binned_activations = {}
mouse_slopes = {}
mouse_taus = {}
mouse_ensembles = {}
mouse_trial_times = {}
## Loop through each mouse
for mouse in tqdm(old_mice):
    ## Create empty dictionaries to store output
    determined_trends = {}
    binned_activations_dict = {}
    slopes_dict = {}
    tau_dict = {}
    trial_times = {}
    ## Load assemblies
    for session in session_list:
        ## Load specific session's assemblies
        assemblies = ica.load_session_assemblies(mouse, assembly_path, session)
        ## Set activation values as act
        act = assemblies.activations.values
        ## Load a specific session's behavior data
        if not across_time:
            aligned_behavior = pd.read_feather(pjoin(behav_path, f'{mouse}_{session}.feat'))
            ## Get which timestamps are part of which trial for the aligned behavior data
            trials = aligned_behavior.trials
            ## Get length of time for each trial
            time_diff = []
            for trial in np.unique(trials):
                ## Subset aligned_behavior by a given trial
                behavior = aligned_behavior.loc[trials == trial]
                ## Get the first and last timestamp to determine the window
                first_timestamp, last_timestamp = behavior.t.to_numpy()[0], behavior.t.to_numpy()[-1]
                ## Convert from ms to s
                first_timestamp = first_timestamp / 1000
                last_timestamp = last_timestamp / 1000
                ## Append to time_diff list
                time_diff.append(last_timestamp - first_timestamp)
            trial_times[session] = time_diff
        ## This is where the data gets binned either by even time intervals or by trials
        if across_time:
          trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_time(act, z_threshold=None, x_bin_size=x_bin_size, analysis_type=analysis_type, 
                                                                                           zscored=True, alpha_old=alpha_old)  
        else:
            ## Define ensemble trends across trials to determine if activation strength is increasing/decreasing across the session
            trends, binned_activations, slopes, tau = ica.define_ensemble_trends_across_trials(act, aligned_behavior, trial_type='all', 
                                                                                               z_threshold=None, alpha_old=alpha_old, analysis_type=analysis_type)
        ## Save to dictionaries
        determined_trends[session] = trends
        binned_activations_dict[session] = binned_activations
        slopes_dict[session] = slopes
        tau_dict[session] = tau
    ## Determine the proportion of ensembles that are increasing, decreasing, or have no trend based on their activation strength across time
    proportion_dict = ica.calculate_proportions_ensembles(determined_trends)
    ## Save to mouse dictionaries before looping to the next mouse
    mouse_trends[mouse] = proportion_dict
    mouse_binned_activations[mouse] = binned_activations_dict
    mouse_slopes[mouse] = slopes_dict
    mouse_taus[mouse] = tau_dict
    mouse_ensembles[mouse] = determined_trends
    mouse_trial_times[mouse] = trial_times

In [ ]:
## Loop through all old mice to determine the proportion of fading ensembles that are matched between sessions
session_one = 'Training4' 
session_two = 'Reversal1'
iterations = 100
direction = 'positive'
alpha = 0.05
percentile = 99
session_of_interest = 'Reversal1'
test = 'pearson'

combined_fading_old = pd.DataFrame()
combined_nonfading_old = pd.DataFrame()
combined_results_old = pd.DataFrame()
for mouse in tqdm(old_mice):
    print(f'Analyzing {mouse}...')
    matched_ensembles = ica.match_ensembles_between_sessions(mouse, registration_path, assembly_path, session_one, session_two,
                                                         direction, test=test, iterations=iterations, alpha=alpha, percentile=percentile)
    matched_ensembles = matched_ensembles[matched_ensembles['statistic'] > 0.2] ## value determined from 99.5th percentile of intra-session correlations
    results_df, fading_ensembles, nonfading_ensembles = ica.determine_proportion_matched(mouse, mouse_ensembles, matched_ensembles, session_of_interest)

    ## Combine results across mice
    combined_fading_old = pd.concat([combined_fading_old, fading_ensembles])
    combined_nonfading_old = pd.concat([combined_nonfading_old, nonfading_ensembles])
    combined_results_old = pd.concat([combined_results_old, results_df])

In [ ]:
## Plot the proportion of matched fading and nonfading ensembles across young mice
fading_results_old = combined_results_old
fading_results_old['prop_fading_matched'] = fading_results_old['num_fading_matched'] / fading_results_old['num_fading']
fading_results_old['prop_nonfading_matched'] = fading_results_old['num_nonfading_matched'] / fading_results_old['num_nonfading']
fading_per_mouse_old = fading_results_old.melt(id_vars=['mouse'], value_vars=['prop_fading_matched', 'prop_nonfading_matched'], 
                                 var_name='ensemble_type', value_name='prop').replace({'prop_fading_matched': 'Fading',
                                                                                       'prop_nonfading_matched': 'Non-fading'})
fading_avg_old = fading_per_mouse_old.groupby(['ensemble_type'], as_index=False).agg({'prop': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Proportion Matched')
fig.add_trace(go.Bar(x=fading_avg_old['ensemble_type'], y=fading_avg_old['prop']['mean'],
                     error_y=dict(type='data', array=fading_avg_old['prop']['sem'], thickness=2.5, width=10),
                     marker_color='darkblue', marker_line_color='black', 
                     marker_line_width=2, opacity=0.8, showlegend=False))
for mouse in np.unique(fading_per_mouse_old['mouse']):
    mouse_data = fading_per_mouse_old[fading_per_mouse_old['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mouse_data['ensemble_type'], y=mouse_data['prop'], mode='markers', 
                             marker_color='darkgrey', name=mouse, showlegend=False, marker=dict(line=dict(width=1))))
fig.show()
# fig.write_image(pjoin(fig_path, 'proportion_matched_R_T4_aged.png'))

In [ ]:
## Combined young and aged mice
fig = pf.custom_graph_template(x_title='', y_title='', shared_y=True, rows=1, columns=2, 
                               titles=['Young', 'Middle Aged'], width=600)
## Young mice
fig.add_trace(go.Bar(x=fading_avg['ensemble_type'], y=fading_avg['prop']['mean'],
                     error_y=dict(type='data', array=fading_avg['prop']['sem'], thickness=2.5, width=10),
                     marker_color='turquoise', marker_line_color='black', 
                     marker_line_width=2, opacity=0.8, showlegend=False), row=1, col=1)
for mouse in np.unique(fading_per_mouse['mouse']):
    mouse_data = fading_per_mouse[fading_per_mouse['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mouse_data['ensemble_type'], y=mouse_data['prop'], mode='markers', 
                             marker_color='darkgrey', name=mouse, showlegend=False, marker=dict(line=dict(width=1))), row=1, col=1)
## Middle-aged mice
fig.add_trace(go.Bar(x=fading_avg_old['ensemble_type'], y=fading_avg_old['prop']['mean'],
                     error_y=dict(type='data', array=fading_avg_old['prop']['sem'], thickness=2.5, width=10),
                     marker_color='darkblue', marker_line_color='black', 
                     marker_line_width=2, opacity=0.8, showlegend=False), row=1, col=2)
for mouse in np.unique(fading_per_mouse_old['mouse']):
    mouse_data = fading_per_mouse_old[fading_per_mouse_old['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mouse_data['ensemble_type'], y=mouse_data['prop'], mode='markers', 
                             marker_color='darkgrey', name=mouse, showlegend=False, marker=dict(line=dict(width=1))), row=1, col=2)
fig.update_yaxes(title='Proportion Matched R to T4', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'young_middle_proportion_R_T4.png'))

### Observe where fading ensembles co-fire

In [4]:
## For a mouse, show where the fading ensembles tend to co-fire
mouse = 'Fornax'
session = 'Reversal1'
assemblies = ica.load_session_assemblies(mouse, assembly_path, session)
behav = pd.read_feather(pjoin(behav_path, f'{mouse}_{session}.feat'))
reward_ports = np.unique(behav.loc[behav['water'] == True, 'lick_port'])
reward_one, reward_two = reward_ports[1], reward_ports[2] ## skip 0 index because it's -1
behav = ctb.fix_lick_ports(behav, reward_one, reward_two)

## Assign coordinates of behavior to assemblies
assemblies = assemblies.assign_coords(behav_t=('frame', behav['t']/1000),
                                    behav_frame=('frame', behav['frame']),
                                    x=('frame', behav['x']),
                                    y=('frame', behav['y']),
                                    lick_port=('frame', behav['lick_port']),
                                    water=('frame', behav['water']),
                                    trials=('frame', behav['trials']),
                                    lin_position=('frame', behav['lin_position']))
## Reward positions for plotting
reward_one_pos = np.mean(assemblies['lin_position'][assemblies['lick_port'] == reward_one])
reward_two_pos = np.mean(assemblies['lin_position'][assemblies['lick_port'] == reward_two])
# ## Calculate place metrics for an ensemble
nbins = 30
alpha = 0.001
place_assembly_data = pc.PlaceFields(x=assemblies['x'].values,
                                    y=assemblies['y'].values,
                                    t=assemblies['behav_t'].values,
                                    neural_data=assemblies['activations'].values,
                                    circular=False,
                                    linearized=True,
                                    shuffle_test=True,
                                    nbins=nbins)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/EnsembleRemodeling_Resubmission/../place_cells.py:44: RuntimeWarning: invalid value encountered in divide
  velocity = dists / np.diff(t, prepend = 0) ## in seconds


Doing shuffle tests. This may take a while.


In [5]:
## Rasters of fading ensembles
rasters, bin_edges = ica.make_ensemble_raster(assemblies, bin_size=0.2, running_only=False, velocity_thresh=7, ensemble_ids=mouse_ensembles['Fornax']['Reversal1']['decreasing'])

In [23]:
fig = pf.plot_ensemble_raster(bin_edges, rasters, ensemble_id=0, normalized=True, reward_positions=[reward_one_pos.values, reward_two_pos.values],
                           x_title='Linearized Position', y_title='Trial')
fig.show()